<a href="https://colab.research.google.com/github/mess-alina/ML/blob/main/Coursework_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Курсовой проект_Воробьева А.Н.**

## ***Машинное обучение в бизнесе. Тема "Вычисление негативных отзывов"***

In [1]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 1.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

#working with text
from sklearn.feature_extraction.text import TfidfVectorizer

#normalizing data
from sklearn.preprocessing import StandardScaler

#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import precision_score,recall_score

#imputer
from sklearn.impute import SimpleImputer

import sklearn.datasets

Загрузим данные фейковые новости или нет

Ссылка на google drive: https://drive.google.com/file/d/1IIlro5YT2RZ0NLWCIdq1U6lZMF-3LDhH/view?usp=sharing

In [8]:
!wget 'https://drive.google.com/uc?export=download&id=1IIlro5YT2RZ0NLWCIdq1U6lZMF-3LDhH' -O FA-KES-Datasets.csv

--2023-07-27 09:17:01--  https://drive.google.com/uc?export=download&id=1IIlro5YT2RZ0NLWCIdq1U6lZMF-3LDhH
Resolving drive.google.com (drive.google.com)... 74.125.143.139, 74.125.143.100, 74.125.143.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.143.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-2o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k0rmjb96hitiff69nd8idkv1tjpp05hp/1690449375000/15969408828065707856/*/1IIlro5YT2RZ0NLWCIdq1U6lZMF-3LDhH?e=download&uuid=48826ae8-0aa4-4d54-9102-96c4f9956021 [following]
--2023-07-27 09:17:02--  https://doc-04-2o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/k0rmjb96hitiff69nd8idkv1tjpp05hp/1690449375000/15969408828065707856/*/1IIlro5YT2RZ0NLWCIdq1U6lZMF-3LDhH?e=download&uuid=48826ae8-0aa4-4d54-9102-96c4f9956021
Resolving doc-04-2o-docs.googleusercontent.com (doc-04-2o-docs.googleusercontent.com)... 108.177.119.132, 2a00

In [10]:
df = pd.read_csv("./FA-KES-Datasets.csv", encoding = 'unicode_escape')
df.head(3)

,unit_id,article_title,article_content,source,date,location,labels
0,1914947530,Syria attack symptoms consistent with nerve ag...,Wed 05 Apr 2017 Syria attack symptoms consiste...,nna,4/5/2017,idlib,0
1,1914947532,Homs governor says U.S. attack caused deaths b...,Fri 07 Apr 2017 at 0914 Homs governor says U.S...,nna,4/7/2017,homs,0
2,1914947533,Death toll from Aleppo bomb attack at least 112,Sun 16 Apr 2017 Death toll from Aleppo bomb at...,nna,4/16/2017,aleppo,0


In [11]:
df['labels'].value_counts()

1    426
0    378
Name: labels, dtype: int64

Дисбаланса классов нет

Разделим данные на train/test и сохраним тестовую выборку на диск

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df, df['labels'], test_size=0.33, random_state=42, stratify=df['labels'])
# save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)

# save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [13]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [14]:
features = ['article_title', 'article_content',	'source',	'location']
target = 'labels'

Соберем кусок, ответственный за feature engineering

In [15]:
# combine
article_title = Pipeline([
                ('imputer', TextImputer('article_title', '')),
                ('selector', ColumnSelector(key='article_title')),
                ('tfidf', TfidfVectorizer())
            ])

article_content = Pipeline([
                ('imputer', TextImputer('article_content', '')),
                ('selector', ColumnSelector(key='article_content')),
                ('tfidf', TfidfVectorizer())
            ])

source = Pipeline([
                ('imputer', TextImputer('source', '')),
                ('selector', ColumnSelector(key='source')),
                ('tfidf', TfidfVectorizer())
            ])

location = Pipeline([
                ('imputer', TextImputer('location', '')),
                ('selector', ColumnSelector(key='location')),
                ('tfidf', TfidfVectorizer())
            ])


feats = FeatureUnion([('article_title', article_title),
                      ('article_content', article_content),
                      ('source', source),
                      ('location', location)])

In [16]:
%%time

pipeline = Pipeline([
    ('features', feats),
    ('classifier', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

CPU times: user 599 ms, sys: 1.67 ms, total: 601 ms
Wall time: 763 ms


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('article_title',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='article_title',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='article_title')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('article_content',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='article_content',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='a...
                                                                  TfidfVectorizer())])),
                                                ('source',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='source',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='source')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())])),
                                                ('location',
                                                 Pipeline(steps=[('imputer',
                                                                  TextImputer(key='location',
                                                                              value='')),
                                                                 ('selector',
                                                                  ColumnSelector(key='location')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('classifier', LogisticRegression())])

In [17]:
with open("logreg_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)

## **Предсказание**

In [18]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [20]:
X_test.head(3)

,unit_id,article_title,article_content,source,date,location,labels
0,1926479005,Deadly Russian Airstrikes Kill 35 & Destroy Ho...,Tuesday 31 May 2016 18:14 Deadly Russian Airst...,etilaf,5/31/2016,idlib,1
1,1965511169,ISIL Terrorists Execute 24 Civilians after Sei...,29-07-2016 ISIL Terrorists Execute 24 Civilian...,manar,7/29/2016,aleppo,1
2,1926479251,135 Civilians Killed in Escalating Bombardment...,Wednesday 07 December 2016 135 Civilians Kille...,etilaf,12/7/2016,idlib,0


In [21]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    pipeline = dill.load(in_strm)

In [22]:
preds = pipeline.predict_proba(X_test)[:, 1]

pred_df = pd.DataFrame({'preds': preds})
pred_df.to_csv("test_predictions.csv", index=None)

In [23]:
preds[:10]

array([0.39822255, 0.62729321, 0.45721468, 0.60268595, 0.70600218,
       0.65746045, 0.56242865, 0.49799596, 0.62359994, 0.42020662])

In [24]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.7742830249226311, F-Score=nan, Precision=0.000, Recall=0.000


<ipython-input-24-4a2c49c824d7>:3: RuntimeWarning: invalid value encountered in true_divide
  fscore = (2 * precision * recall) / (precision + recall)


## **Сервер  FLASK**

In [25]:
!pip install flask-ngrok

In [26]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
import pandas as pd

In [27]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2T9ZEEvd2sEKDJJhycCxZpyiaWT_6EE7yt9HBoKLnPQqFGWpa
!./ngrok http 80

--2023-07-27 11:30:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  12.8MB/s    in 1.0s    

2023-07-27 11:30:35 (12.8 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]

ngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [28]:
with open('logreg_pipeline.dill', 'rb') as in_strm:
    model = dill.load(in_strm)

In [29]:
# Обработчики и запуск Flask
app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run


@app.route("/", methods=["GET"])
def general():
    return "Welcome to prediction process"

@app.route('/predict', methods=['POST'])
def predict():
    data = {"success": False}

    # ensure an image was properly uploaded to our endpoint
    article_title, article_content,	source,	location = "", "", "", ""
    request_json = request.get_json()

    if request_json["article_title"]:
        article_title = request_json['article_title']

    if request_json["article_content"]:
        article_content = request_json['article_content']

    if request_json["source"]:
        source = request_json['source']

    if request_json["location"]:
        location = request_json['location']


    preds = model.predict_proba(pd.DataFrame({"article_title": [article_title],
                                              "article_content": [article_content],
                                              "source": [source],
                                              "location": [location]}))
    data["predictions"] = preds[:, 1][0]
    data["description"] = article_title
        # indicate that the request was a success
    data["success"] = True
    print('OK')

        # return the data dictionary as a JSON response
    return jsonify(data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://9906-34-32-252-239.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [27/Jul/2023 11:51:32] "POST /predict HTTP/1.1" 200 -


OK
